<a href="https://colab.research.google.com/github/revtheundead/financial-agent-study/blob/main/finance_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Goal of this Project:


# Goal of this Project
This project aims to build AI agents for handling different financial tasks efficiently.

## Planned Agents:
1. **Tool Calling Agent** – Uses web search for real-time financial data.  
2. **RAG-based Query Agent** – Retrieves information from a knowledge base for fact-based responses.  
3. **Deep Research Stock Analysis Agent** – Conducts in-depth financial analysis and market research.  
4. **Evaluation Agent (LLM-as-a-Judge)** – Assesses the quality and accuracy of responses from other agents.


# Open-Source Tech Stack
We are using the following open-source technologies:

- **Groq** – Fast inference for large language models ([Link](https://groq.com/))  
- **Agno** – An agentic framework for building AI-driven workflows ([Link](https://www.agno.com/))  
- **PgVector** – A vector database for storing and retrieving financial embeddings ([Link](https://pypi.org/project/pgvector/))  
- **Sentence-Transformers** – Generates high-quality text embeddings for financial analysis ([Link](https://huggingface.co/sentence-transformers))  
- **Udocker** – Lightweight containerization for running Docker in Colab ([Link](https://github.com/drengskapur/docker-in-colab))  
- **DuckDuckGo** – Privacy-focused web search API for retrieving financial news ([Link](https://github.com/duckduckgo))

# Installing Required Packages
This section installs all necessary Python libraries to run the AI agents. It ensures dependencies are met before running the rest of the notebook.

In [ ]:
!pip install groq yfinance agno
!pip install groq duckduckgo-search newspaper4k lxml_html_clean agno
!pip install -U sqlalchemy 'psycopg[binary]' pgvector pypdf agno
!pip install udocker
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.2/474.2 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 19.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 8.8 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=0b3f72a910d5d1782681db5e51dc062a1701126d0432bf030a0e1474d85e43b3
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Set Environment Variables

In [ ]:
import os
from google.colab import userdata

os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['AGNO_API_KEY'] = userdata.get('AGNO_API_KEY')

# And we can check that they are indeed set
print("Groq API key set!" if os.getenv("GROQ_API_KEY") else "Groq API key not set")
print("Agno API key set!" if os.getenv("AGNO_API_KEY") else "AGNO API key not set")

print("API keys have been set!")

Groq API key set!
Agno API key set!
API keys have been set!


## Agent 1: Functional Tool Calling Capability – Web Search

This agent is responsible for retrieving real-time financial data from the web using external tools. It enables dynamic access to the latest stock market trends, company news, and other financial information by leveraging web search capabilities.

### How This Agent Works:

1. **Integrates Web Search Tools**  
   - Uses **DuckDuckGo** as the primary search engine for retrieving financial data.  
   - Employs **Newspaper4k** for extracting and summarizing news articles from relevant sources.  

2. **Uses an AI Model for Query Processing**  
   - The agent is powered by **Groq's Llama 3 (70B parameters)**, a large language model optimized for finance-related reasoning and summarization.  
   - It processes search results, extracts key insights, and presents structured answers.

3. **Handles Financial Research Queries**  
   - When a user submits a query, the agent conducts a web search, retrieves recent articles, and synthesizes the most relevant information into a clear, structured response.

This setup enables real-time financial intelligence by leveraging external sources for up-to-date market insights.

In [ ]:
from textwrap import dedent
from agno.agent import Agent
from agno.models.groq import Groq
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.newspaper4k import Newspaper4kTools

In [ ]:
# Initialize the research agent with advanced financial journalism capabilities
research_agent = Agent(
    model=Groq(id="llama3-70b-8192"),
    tools=[DuckDuckGoTools(), Newspaper4kTools()],
    description=dedent("""\
        You are an elite financial research analyst with expertise in:

        - In-depth investigative research and financial analysis
        - Fact-checking and source validation
        - Data-driven reporting and visualization
        - Expert opinion synthesis and stakeholder insights
        - Market trend identification and predictive analysis
        - Simplification of complex financial topics
        - Ethical reporting with a balanced perspective
        - Global economic and industry-wide contextualization\
    """),
    instructions=dedent("""\
        ## Research Workflow

        ### **1. Research Phase: Gathering Reliable Information**
        - Identify **five** authoritative and recent sources on the given topic.
        - Prioritize **industry reports, financial news, regulatory statements, and expert opinions**.
        - Identify key **stakeholders, companies, policies, and global economic factors**.
        - Use **multiple sources** to cross-check credibility and avoid bias.

        ### **2. Analysis Phase: Evaluating and Synthesizing Data**
        - Extract and verify **critical financial data, market trends, and key insights**.
        - Cross-reference facts from multiple **independent and authoritative** sources.
        - Identify **emerging patterns, contradictions, and potential risks/opportunities**.
        - Compare expert viewpoints and evaluate their credibility based on past accuracy.

        ### **3. Report Compilation: Structuring the Findings**
        - Craft an **engaging, data-backed headline**.
        - Structure the content in **a professional financial report format**.
        - Include **quantitative analysis, statistics, and expert quotes** for credibility.
        - Maintain **objectivity** while providing insightful analysis.
        - Use **clear, precise, and accessible language**, ensuring readability for diverse audiences.

        ### **4. Quality Assurance: Ensuring Accuracy & Clarity**
        - **Verify** all facts, figures, and attributions for credibility.
        - Maintain **logical flow and readability** while avoiding redundant details.
        - Add **context where necessary**, ensuring clarity on financial and economic implications.
        - Discuss **future outlook and potential impacts** on industries, stakeholders, and the economy.

    """),
    expected_output=dedent("""\
        # {Compelling Financial Headline}

        ## Executive Summary
        {Concise, high-level overview of key findings and significance}

        ## Background & Context
        {Historical perspective and relevance}
        {Overview of the current financial landscape}

        ## Key Findings
        {Major discoveries, analysis, and insights}
        {Expert opinions and notable statements}
        {Market data, statistics, and relevant financial trends}

        ## Industry & Economic Impact
        {Effects on companies, investors, and policymakers}
        {Implications for financial markets and global economy}
        {Stakeholder perspectives and risk assessments}

        ## Future Outlook
        {Emerging trends and market predictions}
        {Potential regulatory or policy changes}
        {Challenges and opportunities for businesses and investors}

        ## Expert Insights
        {Key quotes from financial analysts, economists, or industry leaders}
        {Contrasting viewpoints and debates within the sector}

        ## Sources & Research Methodology
        {Comprehensive list of primary sources and their contributions}
        {Research methodology, validation process, and analytical approach}

        ---
        **Research conducted by:** Financial Research Agent
        **Report Type:** Data-Driven Financial Analysis
        **Published:** {current_date}
        **Last Updated:** {current_time}\
    """),
    markdown=True,
    show_tool_calls=True,
    add_datetime_to_instructions=True,
)

# Example prompt 1
research_agent.print_response("Analyze the current state and future implications \
                              of artificial intelligence in Finance", stream=True)


Output()

In [ ]:
# Example prompt 2
research_agent.print_response("Applications of Gen AI in Financial Services", stream=True)


Output()

WARNING  Could not run function duckduckgo_search(query=Applications of Gen AI in Financial Services)

ERROR    https://lite.duckduckgo.com/lite/ 202 Ratelimit                                                           
         Traceback (most recent call last):                                                                        
           File "/usr/local/lib/python3.11/dist-packages/agno/tools/function.py", line 359, in execute             
             self.result = self.function.entrypoint(**entrypoint_args, **self.arguments)                           
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^                           
           File "/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_validate_call.py", line 38, in        
         wrapper_function                                                                                          
             return wrapper(*args, **kwargs)                                                                       
                    ^^^^^^^^^^^^^^^^^^^^^^^^                                                                       
           File "/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_validate_call.py", line 111, in       
         __call__                                                                                                  
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^             
           File "/usr/local/lib/python3.11/dist-packages/agno/tools/duckduckgo.py", line 68, in duckduckgo_search  
             return json.dumps(ddgs.text(keywords=query, max_results=(self.fixed_max_results or max_results)),     
         indent=2)                                                                                                 
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^      
           File "/usr/local/lib/python3.11/dist-packages/duckduckgo_search/duckduckgo_search.py", line 252, in text
             raise DuckDuckGoSearchException(err)                                                                  
         duckduckgo_search.exceptions.DuckDuckGoSearchException: https://lite.duckduckgo.com/lite/ 202 Ratelimit

In [ ]:
# Example prompt 3
research_agent.print_response("AI agents in Financial Services", stream=True)


Output()

## Agent 2: Knowledge Base Query Capability – RAG (Retrieval Augmented Generation)

This agent is designed to retrieve relevant financial knowledge from a structured knowledge base and generate accurate, well-informed responses. It leverages **Retrieval Augmented Generation (RAG)**, a technique that enhances language models by combining search-based retrieval with generative AI.

### How This Agent Works:

1. **Retrieves Information from a Knowledge Base**
   - Uses a **vector database** (such as **PgVector**) to store and index financial documents.
   - Fetches **relevant passages** from stored reports, earnings calls, or market analyses based on the user’s query.

2. **Enhances Accuracy with RAG**
   - Instead of relying solely on an AI model’s pre-trained knowledge, RAG dynamically retrieves up-to-date information before generating responses.
   - Ensures that answers are **fact-based, grounded in real data, and contextually relevant**.

3. **Processes Queries Efficiently**
   - Accepts user questions related to finance, investments, and market trends.
   - Searches the knowledge base for **relevant documents**.
   - Uses an **LLM (such as Groq's Llama 3)** to summarize and generate a structured, easy-to-understand response.

### Why RAG is Important for Financial Analysis:

- Prevents outdated or incorrect information by **retrieving real-world data in real-time**.
- Provides **traceable sources**, allowing users to verify where the information came from.
- Ideal for **handling complex financial reports, earnings transcripts, and market research**.

By combining **retrieval and generation**, this agent ensures **accurate, well-supported answers** to finance-related queries.

Since we are using OS Vector Databases, we need to use Docker to launch the app and initialize the database

In [ ]:
# Copyright 2024 Drengskapur
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
# @title {display-mode:"form"}
# @markdown <br/><br/><center><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/docker.svg" height="150"><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/colab.svg" height="150"></center><br/>
# @markdown <center><h1>Docker in Colab</h1></center><center>github.com/drengskapur/docker-in-colab<br/><br/><br/><b>udocker("run hello-world")</b></center><br/>
def udocker_init():
    import os
    if not os.path.exists("/home/user"):
        !pip install udocker > /dev/null
        !udocker --allow-root install > /dev/null
        !useradd -m user > /dev/null
    print(f'Docker-in-Colab 1.1.0\n')
    print(f'Usage:     udocker("--help")')
    print(f'Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples')

    def execute(command: str):
        user_prompt = "\033[1;32muser@pc\033[0m"
        print(f"{user_prompt}$ udocker {command}")
        !su - user -c "udocker $command"

    return execute

udocker = udocker_init()

Docker-in-Colab 1.1.0

Usage:     udocker("--help")
Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples


Now we need to initialize the PGVector. It's simpler if you are running from your local terminal but to make it work in the Colab, we need to run the below steps:

In [ ]:
# 1. First install udocker
!udocker --allow-root install

# 2. Kill existing processes and clean up
!pkill -9 -f postgres
!rm -rf /content/pgdata
!udocker --allow-root rm pgvector
!rm -f postgres.log

# 3. Create fresh directory
!mkdir -p /content/pgdata
!chmod -R 777 /content/pgdata

# 4. Pull and create container with correct image path
!udocker --allow-root pull ankane/pgvector
!udocker --allow-root create --name=pgvector ankane/pgvector

# 5. Run the container
!nohup udocker --allow-root run \
    --env="POSTGRES_DB=ai" \
    --env="POSTGRES_USER=ai" \
    --env="POSTGRES_PASSWORD=ai" \
    --env="PGDATA=/var/lib/postgresql/data/pgdata" \
    --volume="/content/pgdata:/var/lib/postgresql/data" \
    --publish="5532:5432" \
    pgvector > postgres.log 2>&1 &

# 6. Connection testing
import time
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError

def test_db_connection(max_retries=5, wait_time=10):
    db_url = "postgresql+psycopg://ai:ai@localhost:5532/ai"

    for attempt in range(max_retries):
        try:
            print(f"\nConnection attempt {attempt + 1}/{max_retries}")
            engine = create_engine(db_url)
            with engine.connect() as connection:
                result = connection.execute(text("SELECT version();"))
                version = result.fetchone()[0]
                print("✅ Successfully connected to PostgreSQL!")
                print(f"Server Version: {version}")

                # Test vector extension
                connection.execute(text("CREATE EXTENSION IF NOT EXISTS vector;"))
                print("✅ Vector extension ready!")
                return engine
        except OperationalError as e:
            print(f"Attempt {attempt + 1} failed, waiting {wait_time} seconds...")
            print("\nChecking postgres status:")
            !ps aux | grep postgres
            print("\nLatest logs:")
            !tail -n 20 postgres.log
            time.sleep(wait_time)

    return None

# 7. Apply 30 secs sleep time to wait for DB to finish set up before testing for connection
print("Waiting for database to initialize...")
time.sleep(30)

engine = test_db_connection()

if engine:
    print("\n✅ Database is ready for RAG Agent initialization!")
else:
    print("\n❌ Database connection failed. Please check the logs above.")

Error: invalid container id 
Info: downloading layer sha256:f2c967e41f72b294e2b96f25154dda38dbde3603b3be33888fb437147972f24b
Info: downloading layer sha256:c5f09b50002256f9e40253d9f3f34381edbe3ca083eb5ce77ecffc874c087995
Info: downloading layer sha256:2e3723549f1143b2c0381181709301932d6a592d8969d0827c1f0133772dfbe0
Info: downloading layer sha256:7077e54346e0cc4692391042abd0479bb02443892be7c6b1085fe7184caff826
Info: downloading layer sha256:bb153abf380255875eda2f78bb3c853520a77f3175574a91d909b5d6912c75a4
Info: downloading layer sha256:f1a157d7d7b01f004e4e758a97a38a5d10c8ce79348e5b674187a99d4f0cabda
Info: downloading layer sha256:6e662fa63f18991e2026f333e95c9670506a0c891ec82e5593bb613a627c6a96
Info: downloading layer sha256:2c35234636c95a2fed252512bb033c920753cffdd75c796da556a594845c121d
Info: downloading layer sha256:04efcdd3a2a4cbfcbdd1542bb9af0b2ff422f4e7b2cde58bfe8c61521df96056
Info: downloading layer sha256:786562b3be85b223d9577821b409d7147981e9b2c9611e0c5ec8725b1255df43
Info: downl

## Document QA Class

This section defines the **Document QA** class, which is responsible for handling **question-answering (QA) tasks** over financial documents.

### Functionalities:

1. **Initialize an Open-Source (OS) Sentence Transformer Model**
   - Loads a **sentence-transformer embedding model** to process text.
   - Converts documents into numerical vector representations for efficient retrieval.

2. **Access the Vector Database (Vector DB)**
   - Connects to a **PgVector database** that stores document embeddings.
   - Loads the embeddings corresponding to the **PDF URL provided by the user**.

3. **Initialize the Database and QA System**
   - Sets up the database connection and prepares it for **querying financial documents**.
   - Ensures the system is ready to perform **retrieval-augmented question-answering (RAG)**.

In [ ]:
from typing import Union, List, Tuple
from agno.vectordb.pgvector import PgVector
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase
from sentence_transformers import SentenceTransformer

class DocumentQA:
    def __init__(self):
        # Initialize embedder
        self.embedder = self._create_embedder()
        # Initialize Groq model
        self.chat_model = Groq(id="llama3-8b-8192")
        # Database URL
        self.db_url = "postgresql+psycopg://ai:ai@localhost:5532/ai"
        self.current_knowledge_base = None
        self.agent = None

    def _create_embedder(self):
        """Create the embedding model"""
        class EmbeddingModel:
            def __init__(self):
                self.model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')
                self.dimensions = 384

            def get_embedding_and_usage(self, text: Union[str, List[str]]) -> Tuple[Union[List[List[float]], List[float]], dict]:
                if isinstance(text, str):
                    embedding = self.model.encode(text)
                    embedding_list = embedding.tolist()
                    usage = {"prompt_tokens": len(text.split()), "total_tokens": len(text.split())}
                    return embedding_list, usage

                embeddings = self.model.encode(text)
                embedding_list = embeddings.tolist()
                total_tokens = sum(len(t.split()) for t in text)
                usage = {"prompt_tokens": total_tokens, "total_tokens": total_tokens}
                return embedding_list, usage

            def get_embedding(self, text: Union[str, List[str]]) -> Union[List[float], List[List[float]]]:
                return self.model.encode(text).tolist()

        print("✅ Embedding model(sentence-transformers/paraphrase-MiniLM-L6-v2) initialized successfully!")
        return EmbeddingModel()

    def load_pdf_url(self, url: str, table_name: str = "documents"):
        """Load a PDF from a URL"""
        try:
            # Create PDF URL knowledge base
            self.current_knowledge_base = PDFUrlKnowledgeBase(
                urls=[url],
                vector_db=PgVector(
                    table_name=table_name,
                    db_url=self.db_url,
                    embedder=self.embedder
                ),
            )

            # Initialize the Agent
            self.agent = Agent(
                knowledge=self.current_knowledge_base,
                search_knowledge=True,
                model=self.chat_model
            )

            # Load knowledge base
            print("Loading knowledge base...")
            self.current_knowledge_base.load(recreate=True)
            print("✅ Knowledge base loaded successfully!")

            # Show sample content
            self.show_sample_content()

        except Exception as e:
            print(f"❌ Error loading PDF: {e}")
            import traceback
            print(traceback.format_exc())

    def show_sample_content(self, num_samples: int = 5):
        """Show sample content from the knowledge base"""
        try:
            if not self.current_knowledge_base:
                print("No knowledge base loaded!")
                return

            docs = self.current_knowledge_base.search("")
            print("\nSample documents in knowledge base:")
            print("-" * 50)
            for i, doc in enumerate(docs[:num_samples], 1):
                print(f"\nDocument {i}:")
                if hasattr(doc, 'content'):
                    print(doc.content[:200] + "..." if len(doc.content) > 200 else doc.content)
                elif hasattr(doc, 'text'):
                    print(doc.text[:200] + "..." if len(doc.text) > 200 else doc.text)
        except Exception as e:
            print(f"Error showing samples: {e}")

    def ask(self, question: str):
        """Ask a question about the loaded document"""
        if not self.current_knowledge_base or not self.agent:
            print("Please load a document first!")
            return

        print(f"\nQ: {question}")
        try:
            # Get relevant documents
            relevant_docs = self.current_knowledge_base.search(question)
            print("\nRelevant documents found:", len(relevant_docs) if relevant_docs else 0)

            # Build context from relevant documents
            context = "\n".join([doc.content if hasattr(doc, 'content') else doc.text
                               for doc in relevant_docs])

            # Create a prompt that includes the context
            full_prompt = f"""Based on the following content:{context}
            Question: {question}
            Please provide a detailed answer based ONLY on the information provided above."""

            # Get response with context
            response = self.agent.run(full_prompt)
            print(f"\nA: {response.content}")

        except Exception as e:
            print(f"Error: {e}")
            import traceback
            print(traceback.format_exc())

In [ ]:
RAG_qa = DocumentQA()

✅ Embedding model(sentence-transformers/paraphrase-MiniLM-L6-v2) initialized successfully!


In [ ]:
RAG_qa.load_pdf_url("https://www.apple.com/environment/pdf/Apple_Environmental_Progress_Report_2024.pdf")

Loading knowledge base...


INFO     Dropping collection

INFO     Table 'ai.documents' does not exist.

INFO     Creating collection

INFO     Loading knowledge base

INFO     Reading: https://www.apple.com/environment/pdf/Apple_Environmental_Progress_Report_2024.pdf

INFO     Inserted batch of 100 documents.

INFO     Inserted batch of 31 documents.

INFO     Added 131 documents to knowledge base

✅ Knowledge base loaded successfully!

Sample documents in knowledge base:
--------------------------------------------------

Document 1:
Environmental progress can and should be good for business. We underpin our climate strategy with strong business principles and innovation while harnessing the power of markets to replicate our solut...

Document 2:
Supporting communities worldwide Through our engagement efforts, we work directly with groups and individuals who are addressing environmental injustice in their communities. We evaluate each opportun...

Document 3:
 Environmental Progress Report 104Engagement and AdvocacyEnvironmental Initiatives DataIntroduction Contents Appendix

Document 4:
      2024 Environmental Progress Report 102Engagement and AdvocacyEnvironmental Initiatives DataIntroduction Contents Appendix

Document 5:
An ambitious goal for 2030: We committed to be carbon neutral for our entire carbon footprint by the end of the decade. Our journey to 2030 centers on

In [ ]:
RAG_qa.ask("Key points in this report? Give in 5 bullets")


Q: Key points in this report? Give in 5 bullets

Relevant documents found: 5

A: Here are 5 key points extracted from the report:

• **Rigorous chemical safety requirements**: Apple has established a Regulated Substances Specification, which sets requirements for the use of chemicals or materials in products, accessories, manufacturing processes, and packaging. The specification builds on Apple's history of advancements in material safety and is updated regularly to reflect the latest scientific research and standards.

• **Chemical restrictions**: Apple has added perfluorohexanesulfonic acid (PFHxA), phenol, isopropylated, phosphate (3:1) (PIP 3:1), and several skin-sensitizing substances to its restricted list, surpassing current regulatory restrictions.

• **Assessment and testing**: Apple evaluates the safety of its products and materials through chemical analyses at its Environmental Testing Lab, using methods such as toxicological assessments and biocompatibility testing.

• **D

In [ ]:
RAG_qa.ask("Executive Summary in 100 words")


Q: Executive Summary in 100 words

Relevant documents found: 5

A: EXECUTIVE SUMMARY

Apple's Environmental Progress Report for 2023 reports on the company's environmental sustainability efforts and achievements for the fiscal year 2022 reporting period (September 25, 2022 through September 30, 2023). The report highlights Apple's progress in reducing its environmental impact, including a decrease in greenhouse gas emissions and an increase in renewable energy usage. The company also reports on its efforts to reduce paper waste and improve water conservation. The report is assured by independent third-party auditors, Apex Companies, LLC, who verified the accuracy and reliability of the environmental data reported by Apple.


## Agent 3: Stock Market Analysis

This agent is designed to perform **comparative stock market analysis** and generate **financial reports** based on real-time and historical data.

### Functionalities:

1. **Utilizes Yahoo Finance for Market Data**
   - Fetches stock price information, analyst recommendations, and historical trends.
   - Compares multiple stocks to identify patterns and market movements.

2. **Performs Comparative Analysis**
   - Evaluates key financial indicators such as **P/E ratio, market cap, revenue growth, and volatility**.
   - Benchmarks stocks against industry averages and competitors.

3. **Generates a Summary Report**
   - Creates a concise report with key financial insights.
   - Highlights trends, potential investment risks, and opportunities.
   - Provides a forward-looking analysis based on market conditions.

This agent may help investors and analysts make **data-driven financial decisions** by automating market research and summarization.

In [ ]:
from textwrap import dedent

from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.yfinance import YFinanceTools

# Initialize the stock market analysis agent with financial intelligence tools
stock_agent = Agent(
    model=Groq(id="llama3-70b-8192"),
    tools=[
        YFinanceTools(
            stock_price=True,
            analyst_recommendations=True,
            stock_fundamentals=True,
            historical_prices=True,
            company_info=True,
            company_news=True,
        )
    ],
    instructions=dedent("""\
        ## Stock Market Analysis & Credit Rating Assessment

        You are an experienced credit rating analyst with a strong background in financial markets, risk evaluation, and equity analysis. Your goal is to provide detailed, data-driven insights while ensuring clarity and depth.

        ---

        ## Step-by-Step Analysis Process

        ### 1. Market Snapshot & Performance Overview
        - Current stock price and recent price movement
        - 52-week high and low for context
        - Daily trading volume and liquidity trends

        ### 2. Financial Health & Fundamentals
        - Key valuation metrics (P/E ratio, Market Cap, EPS, Dividend Yield)
        - Revenue and earnings trends (YoY growth, quarterly changes)
        - Debt levels and financial leverage (Debt-to-equity ratio, cash reserves)
        - Profitability indicators (Net margin, ROE, ROA)

        ### 3. Industry Context & Competitive Positioning
        - Sector trends and market conditions
        - Competitor benchmarking (Compare key financials with industry peers)
        - Sentiment analysis (Analyst ratings, news sentiment)

        ### 4. Risk & Regulatory Considerations
        - Potential risk factors (Market volatility, geopolitical events, company-specific risks)
        - Regulatory environment (Compliance concerns, industry regulations)

        ### 5. Forward-Looking Analysis & Investment Outlook
        - Growth projections (Company’s future potential based on historical trends)
        - Analyst recommendations (Buy/Hold/Sell consensus)
        - Macroeconomic influences (Interest rates, inflation, sector performance)

        ---

        ## Reporting Guidelines
        - Start with an Executive Summary (Concise, key takeaways upfront)
        - Use tables and charts for financial data
        - Structure content with clear section headers
        - Highlight key insights and comparisons using bullet points
        - Explain technical terms in layman’s terms where necessary
        - End with a forward-looking investment perspective

        ---

        ## Risk Disclosure
        - Clearly highlight potential risk factors
        - Address market uncertainties and volatility
        - Mention regulatory concerns relevant to the industry

        ---

        Ensure all information is accurate, up-to-date, and derived from reliable sources.
    """),
    add_datetime_to_instructions=True,
    show_tool_calls=True,
    markdown=True,
)

print("Stock Agent created. Ready to take user queries..")

Stock Agent created. Ready to take user queries..


In [ ]:
# Example query 1
stock_agent.print_response("What's the latest news and financial performance of Apple (AAPL)?", stream=True)

Output()

In [ ]:
# Example query 2: Semiconductor market analysis
stock_agent.print_response(
    dedent("""\
      Analyze the semiconductor market performance focusing on:
      - NVIDIA (NVDA)
      - AMD (AMD)
      - Intel (INTC)
      - Taiwan Semiconductor (TSM)
      Compare their market positions, growth metrics, and future outlook in terms of AI growth.
    """),
    stream=True
)

Output()

In [ ]:
# Example query 3: Competitive analysis

stock_agent.print_response("How is Microsoft performing in the age of AI?", stream=True)

Output()

## Agent 4: Evaluation – LLM-as-a-Judge

This agent is responsible for **evaluating the quality and accuracy** of responses generated by other agents. It acts as a **neutral judge**, ensuring that financial insights and analyses are well-grounded and reliable.

### Key Functionalities:

1. **Assessing Response Accuracy**
   - Evaluates whether the answers provided by other agents are factually correct.
   - Cross-checks responses against financial data sources and market trends.

2. **Scoring Response Quality**
   - Measures clarity, coherence, and depth of analysis in generated responses.
   - Identifies missing information or inconsistencies.

3. **Ensuring Reliability in Financial Reporting**
   - Detects potential bias or misleading conclusions.
   - Provides feedback to improve the accuracy and usefulness of financial insights.

This agent helps maintain **high standards in financial research and analysis** by ensuring that all generated insights meet strict quality benchmarks.

In [ ]:
from textwrap import dedent
from agno.agent import Agent
from agno.models.groq import Groq

class RAGEvaluator:
    def __init__(self):
        self.evaluator = self._initialize_evaluator()

    def _initialize_evaluator(self):
        return Agent(
            model=Groq(id="llama-3.1-8b-instant"),  # Using different Llama model
            description=dedent("""\
                You are an expert RAG system evaluator with deep expertise in:
                - Information retrieval quality assessment
                - Response accuracy evaluation
                - Source attribution verification
                - Context relevance analysis
                - Natural language generation evaluation
            """),
            instructions=dedent("""\
                Evaluate the RAG system output based on these key metrics:

                1. Faithfulness (1-5):
                   - How accurately does the response reflect the source documents?
                   - Are there any hallucinations or incorrect statements?
                   - Does it maintain factual consistency?

                2. Context Relevance (1-5):
                   - Are the retrieved passages relevant to the query?
                   - Is important context missing?
                   - Is irrelevant information included?

                3. Answer Completeness (1-5):
                   - Does the response fully address the query?
                   - Are all key aspects covered?
                   - Is the level of detail appropriate?

                4. Source Attribution (1-5):
                   - Are sources properly cited?
                   - Is it clear which information comes from where?
                   - Can claims be traced back to sources?

                5. Response Coherence (1-5):
                   - Is the response well-structured?
                   - Does it flow logically?
                   - Is it easy to understand?

                Provide specific examples and explanations for each score.
            """),
            expected_output=dedent("""\
                # RAG Evaluation Report

                ## Overview
                Query: {query}
                Response Length: {n_chars} characters

                ## Metric Scores

                ### Faithfulness: {score}/5
                - Justification:
                - Examples:
                - Areas for Improvement:

                ### Context Relevance: {score}/5
                - Justification:
                - Examples:
                - Areas for Improvement:

                ### Answer Completeness: {score}/5
                - Justification:
                - Examples:
                - Areas for Improvement:

                ### Source Attribution: {score}/5
                - Justification:
                - Examples:
                - Areas for Improvement:

                ### Response Coherence: {score}/5
                - Justification:
                - Examples:
                - Areas for Improvement:

                ## Overall Score: {total}/25

                ## Key Recommendations
                1. {rec1}
                2. {rec2}
                3. {rec3}

                ## Summary
                {final_assessment}
            """),
            markdown=True,
        )

    def evaluate(self, query: str, response: str, context: list, stream: bool = True):
        """
        Evaluate a RAG system's response

        Args:
            query (str): Original user query
            response (str): RAG system's response
            context (list): Retrieved passages used for the response
            stream (bool): Whether to stream the evaluation output
        """
        evaluation_prompt = f"""
        Please evaluate this RAG system output:

        QUERY:
        {query}

        RETRIEVED CONTEXT:
        {' '.join(context)}

        RESPONSE:
        {response}

        Provide a detailed evaluation following the metrics and format specified.
        """

        return self.evaluator.print_response(evaluation_prompt, stream=stream)


# Initialize evaluator
evaluator = RAGEvaluator()
print("LLM-as-a Judge Evaluator initialized successfully!")

LLM-as-a Judge Evaluator initialized successfully!


In [ ]:
# Example evaluation. Rerun this to use actual financial RAG outputs

query = "What are the key features of transformer models?"
context = [
    "Transformer models use self-attention mechanisms to process input sequences.",
    "Key features include parallel processing and handling of long-range dependencies."
]
response = "Transformer models are characterized by their self-attention mechanism..."

# Run evaluation
evaluator.evaluate(query, response, context)

Output()

# Appendix

## Papers:
1. **The Rise and Potential of Large Language Model Based Agents: A Survey**  
   [Read here](https://arxiv.org/pdf/2309.07864)  
   - Explores the evolution and capabilities of LLM-based agents, discussing their applications and future potential.

2. **Self-Reflection in LLM Agents: Effects on Problem-Solving Performance**  
   [Read here](https://arxiv.org/pdf/2405.06682v3)  
   - Investigates how self-reflection mechanisms impact the performance and reasoning abilities of LLM agents.

3. **Agent Laboratory: Using LLM Agents as Research Assistants**  
   [Read here](https://arxiv.org/pdf/2501.04227v1)  
   - Analyzes the effectiveness of LLM agents in assisting researchers by automating literature reviews and summarization tasks.

4. **Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks**  
   [Read here](https://arxiv.org/pdf/2005.11401)  
   - Discusses the RAG framework, combining information retrieval and generative models to enhance factual accuracy in AI-generated responses.